In [1]:
import faiss
import pandas as pd
from sentence_transformers import SentenceTransformer

c:\Users\Admin\OneDrive - Association Digital Fabrication Laboratory\Desktop\Patents-code\LLM-Patents\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
import faiss
import gc

# ❌ Step 1: Delete the old index if it exists
try:
    del index  # or whatever your variable is called
except NameError:
    pass  # index wasn't defined

# 🧹 Step 2: Force garbage collection
gc.collect()



2829

In [2]:
index = faiss.read_index("E:\\Meliha-Patenti\\index\\faiss_index_ivf_cosine.bin")
#index = faiss.read_index("E:\\Meliha-Patenti\\index\\faiss_index_ivf_l2.bin")
metadata = pd.read_csv("E:\\Meliha-Patenti\\metadata_all.csv", low_memory=False)

In [3]:
model = SentenceTransformer(".\\local_minilm_l6_v2")  

In [4]:
import pandas as pd

# Load your CSV
df = pd.read_csv("check1.csv")

# Set top-k
k = 10

# Assume model, index, metadata are already loaded
retrieved_ids = []
found_itself_flags = []

for i, row in df.iterrows():
    claim = row['claim']
    true_patent_id = str(row['patent_id']).strip()

    if pd.isna(claim) or not claim.strip():
        retrieved_ids.append("")
        found_itself_flags.append("N")
        continue

    # Increase initial search range to allow deduplication
    search_k = k * 5  # e.g., 50 for top 10 unique
    query_embedding = model.encode([claim], normalize_embeddings=True)
    distances, indices = index.search(query_embedding.astype("float32"), search_k)
    
    # Fetch patent_ids from metadata
    top_k_metadata = metadata.iloc[indices[0]]
    all_ids = top_k_metadata['patent_id'].astype(str)

    # Deduplicate while preserving order, then take top-10 unique
    unique_ids = list(dict.fromkeys(all_ids))[:k]
    
    retrieved_ids.append(" | ".join(unique_ids))
    found_itself_flags.append("Y" if true_patent_id in unique_ids else "N")


# Update the DataFrame
df['top 10 retrieved patents'] = retrieved_ids
df['found itself (Y/N)'] = found_itself_flags

# Save output
df.to_csv("check1_with_retrieved_cos.csv", index=False)

print("✅ Retrieval done. Check 'found itself (Y/N)' column in 'check1_with_retrieved.csv'.")


✅ Retrieval done. Check 'found itself (Y/N)' column in 'check1_with_retrieved.csv'.


In [5]:
import numpy as np
print(np.__version__)

2.3.1


In [1]:
k = 10  # top-k results per query

print("\n🔍 Interactive Patent Search")
print("Type your claim query. Type 'exit' to quit.\n")

while True:
    query = input("📝 Enter your patent claim:\n> ").strip()
    
    if query.lower() in ["exit", "quit"]:
        print("👋 Exiting search.")
        break
    if not query:
        continue

    # Embed and search
    query_embedding = model.encode([query], normalize_embeddings=True)
    distances, indices = index.search(query_embedding.astype("float32"), k)
    top_k_metadata = metadata.iloc[indices[0]]

    # Print results
    print("\n📄 Top-k Results:")
    for rank, (row, dist) in enumerate(zip(top_k_metadata.itertuples(index=False), distances[0]), 1):
        print(f"\nRank {rank}")
        print(f"Distance: {dist:.4f}")
        print(f"Patent ID: {row.patent_id}")
        print(f"Claim: {row.cleaned_claim}")  # adjust column name if needed
        print("-" * 50)

    print("\n🌟 You can now enter another query or type 'exit' to stop.\n")


🔍 Interactive Patent Search
Type your claim query. Type 'exit' to quit.



NameError: name 'model' is not defined